In [ ]:
import sys
import os
import lzma
import random
from collections import defaultdict
import math

In [ ]:
import numpy
import pandas

In [ ]:
import xgboost as xgb

In [ ]:
import sklearn

In [ ]:
from sklearn.metrics import recall_score, precision_score, f1_score, confusion_matrix

In [ ]:
from catboost import Pool, CatBoostClassifier
import catboost

In [ ]:
treatment_columns = ['tumor_size_cm_preTrt_preSurgery', 
                     'tumor_size_cm_secondAxis_preTrt_preSurgery', 
                     'preTrt_lymph_node_status', 
                     'preTrt_totalLymphNodes', 
                     'preTrt_numPosLymphNodes', 
                     'hist_grade', 
                     'nuclear_grade_preTrt', 
                     'age', 'race', 'menopausal_status', 'surgery_type', 'intarvenous', 'intramuscular', 'oral', 
                     'radiotherapyClass', 'chemotherapyClass', 'hormone_therapyClass', 'postmenopausal_only',
                     'immediate_biol_target', 'anthracycline', 'taxane', 'anti_estrogen', 'aromatase_inhibitor',
                     'estrogen_receptor_blocker', 'estrogen_receptor_blocker_and_stops_production', 
                     'estrogen_receptor_blocker_and_eliminator', 'anti_HER2', 
                     'tamoxifen', 'doxorubicin', 
                     'epirubicin', 'docetaxel', 'capecitabine', 'fluorouracil',
                     'paclitaxel', 'cyclophosphamide', 'anastrozole', 
                     'fulvestrant', 'gefitinib', 'trastuzumab', 'letrozole', 'chemotherapy', 'hormone_therapy',
                     'no_treatment', 'methotrexate', 'cetuximab', 'carboplatin', 'other', 'taxaneGeneral']

In [ ]:
cancer_data_dir = '/home/noskill/projects/cancer/data'
dump_dir = os.path.join(cancer_data_dir, 'bcDump/example15bmc')
clinical_table_path = os.path.join(cancer_data_dir, 'bcClinicalTable.csv')
merged_path = os.path.join(dump_dir, 'ex15bmcMerged.csv.xz')
bmc_all_path = os.path.join(dump_dir, 'bmc15mldata1.csv')

In [ ]:
dtype = {'DFS': pandas.Int64Dtype(),
         'pCR': pandas.Int64Dtype(),
         'RFS': pandas.Int64Dtype(), 
         'DFS': pandas.Int64Dtype(), 
         'posOutcome': pandas.Int64Dtype()}

# Convertors for mapping string data to numbers

In [ ]:
def convert_surgery(x, surgery_mapping=dict()):
    if x not in surgery_mapping:
        surgery_mapping[x] = len(surgery_mapping) + 1
    return surgery_mapping[x]


def convert_node_status(x, mapping=dict()):
    if x == 'NA' or x == 'NaN':
        return numpy.nan
    if not isinstance(x, str) and numpy.isnan(x):
        return x
    if x not in mapping:
        mapping[x] = len(mapping) + 1
    return mapping[x]


def convert_race(x, mapping=dict()):
    return convert_node_status(x, mapping)

def convert_menapause(x, mapping=dict()):
    return convert_node_status(x, mapping)

converters=dict(preTrt_lymph_node_status=convert_node_status,
               race=convert_race,
               menopausal_status=convert_menapause,
               surgery_type=convert_surgery,
               surgery=convert_surgery)

# load averaged treatment table

In [ ]:
bmc = pandas.read_csv(bmc_all_path, dtype=dtype, converters=converters)
bmc = bmc.sort_values(by='patient_ID')

# load detailed treatment

In [ ]:
treatment = pandas.read_csv(clinical_table_path, converters=converters).sort_values(by='patient_ID')
treatment = treatment[treatment.patient_ID.isin(bmc.patient_ID)]

In [ ]:
bmc.head()

# load genes expression data

In [ ]:
gene_expression = pandas.read_csv(lzma.open(merged_path))

In [ ]:
gene_expression.head(5)

In [ ]:
genes_features = gene_expression[gene_expression.patient_ID.isin(bmc.patient_ID)]

In [ ]:
genes_features = genes_features.sort_values(by='patient_ID')

# columns to use for training

In [ ]:
aggregated_treatment_columns = ['radio', 'surgery', 'chemo', 'hormone']
label_columns = ['pCR', 'RFS', 'DFS', 'posOutcome']
label_columns = ['posOutcome']
feature_columns = genes_features.columns.to_list()[1:] + treatment_columns

## merge genes expression + averaged treatment + detailed treatment

In [ ]:
merged = pandas.merge(genes_features, bmc, left_on='patient_ID', right_on='patient_ID')
merged = pandas.merge(merged, treatment, left_on='patient_ID', right_on='patient_ID')

In [ ]:
def split_by_study(study_name=None):
    """
    Split one study out for cross-validation
    """
    for eval_study in set(bmc.study):
        if study_name:
            eval_study = study_name
        print(eval_study)
        bmc_train = bmc[bmc.study != eval_study]
        bmc_val = bmc[bmc.study == eval_study]
        assert (not set(bmc_train.patient_ID).intersection(set(bmc_val.patient_ID)))

        train_split = merged[merged.patient_ID.isin(bmc_train.patient_ID)]
        val_split = merged[merged.patient_ID.isin(bmc_val.patient_ID)]
        assert val_split.patient_ID.to_list() == bmc_val.patient_ID.to_list()
        train_data = train_split[feature_columns].to_numpy()
        train_labels = train_split[label_columns].to_numpy().astype(int)
        val_data = val_split[feature_columns].to_numpy()
        val_labels = val_split[label_columns].to_numpy().astype(int)
        yield train_data, train_labels, val_data, val_labels
        if study_name:
            break

In [ ]:
def select_balanced_idx(study, num):
    if not num % 2 == 0:
        num = num + 1
    validation = []
    pos_outcome = study[study.posOutcome == 1].patient_ID
    neg_outcome = study[study.posOutcome == 0].patient_ID
    pos_idx = numpy.arange(len(pos_outcome))
    neg_idx = numpy.arange(len(neg_outcome))
    random.shuffle(pos_idx)
    random.shuffle(neg_idx)
    i = 0
    while not (len(validation) >= num):
        validation.append(pos_outcome.iloc[pos_idx[i]])
        validation.append(neg_outcome.iloc[neg_idx[i]])
        i += 1
    train = study[~study.patient_ID.isin(validation)]
    validation = study[study.patient_ID.isin(validation)]
    return train, validation

In [ ]:
def random_split(ratio=0.1, study_name=None, rand=False):
    """
    Split dataset into train and validation sets:
    --------------
    Returns: train_data, train_labels, val_data, val_labels, expected
        expected - confusion matrix expected from classification by ratio of positive/negative for each study
    """
    val_patients = []
    train_patients = []
    expected = dict()
    expected['TN'] = 0
    expected['FN'] = 0
    expected['FP'] = 0
    expected['TP'] = 0
    for eval_study in set(bmc.study):
        if study_name is not None:
            if study_name != eval_study:
                continue
        study = bmc[bmc.study == eval_study]
        num_select = math.ceil(len(study) * ratio)
        study_patients = bmc[bmc.study == eval_study]
        bmc_train, bmc_val = select_balanced_idx(study_patients, num_select)
        pos_prob_train = bmc_train.posOutcome.sum() / len(bmc_train)
        neg_prob_train = 1 - pos_prob_train
        P = bmc_val.posOutcome.sum()
        N = len(bmc_val) - P
        TN = N * neg_prob_train
        TP= P * pos_prob_train
        FP = N - TN
        FN = P - TP
        expected['TN'] += TN
        expected['TP'] += TP
        expected['FP'] += FP
        expected['FN'] += FN
        val_patients += bmc_val.patient_ID.to_list()
        train_patients += bmc_train.patient_ID.to_list()
        
    train_split = merged[merged.patient_ID.isin(train_patients)]
    val_split = merged[merged.patient_ID.isin(val_patients)]
    train_data = train_split[feature_columns].to_numpy()
    train_labels = train_split[label_columns].to_numpy().astype(int).ravel()
    val_data = val_split[feature_columns].to_numpy()
    val_labels = val_split[label_columns].to_numpy().astype(int).ravel()
    if rand:
        train_data = numpy.random.randn(*train_data.shape)
        val_data = numpy.random.randn(*val_data.shape)
    return train_data, train_labels, val_data, val_labels, expected

In [ ]:
def compute_metrics(result, y_true, y_pred, x_true, x_pred):
    result['recall'].append(recall_score(y_true, y_pred))
    result['precision'].append( precision_score(y_true, y_pred))
    result['f1'].append(f1_score(y_true, y_pred))
    result['confusion'].append(confusion_matrix(y_true, y_pred))
    result['train_f1'].append(f1_score(x_true, x_pred))
    result['train_confusion'].append(confusion_matrix(x_true, x_pred))
    confusion = result['confusion'][-1]
    accuracy = (confusion[0][0] + confusion[1][1]) / (sum(confusion[0]) + sum(confusion[1]))
    result['accuracy'].append(accuracy)

# catboost

In [ ]:
res = defaultdict(list)
model = CatBoostClassifier(iterations=3600,
                           depth=4,
                           use_best_model=True,
                           learning_rate=0.015,
                           loss_function='Logloss',
                           model_size_reg=2,
                           verbose=True,
                           scale_pos_weight=0.605,
                           l2_leaf_reg=2,
                           od_type='Iter', od_wait=200)
train_data, train_labels, val_data, val_labels, expected = random_split()
catboost_pool = Pool(train_data, 
                    train_labels)

test_data = Pool(val_data,
                 val_labels) 
# train the model
clf = model.fit(train_data, train_labels, 
          eval_set=test_data,
          save_snapshot=False, snapshot_file='vasya')
y_pred = clf.predict(val_data)
x_pred = clf.predict(train_data)
compute_metrics(res, val_labels.flatten(), y_pred, train_labels, x_pred)
res

# SVM

In [ ]:
from sklearn import datasets, svm, metrics
svm_total = defaultdict(list)
model = svm.SVC(C=1, kernel='rbf', class_weight={1: 0.5})
train_data, train_labels, val_data, val_labels, expected = random_split()
# train the model
clf = model.fit(numpy.nan_to_num(train_data), numpy.nan_to_num(train_labels))
y_pred = clf.predict(numpy.nan_to_num(val_data))
x_pred = clf.predict(numpy.nan_to_num(train_data))
compute_metrics(svm_total, val_labels.flatten(), y_pred, train_labels, x_pred)
for key in svm_total:
    print('{0}: {1}'.format(key, svm_total[key][-1]))

In [ ]:
print(train_data.shape)
print(val_data.shape)

# SVM - single study

In [ ]:
from sklearn import datasets, svm, metrics
svm_total = defaultdict(list)

for study in set(bmc.study):
    print(study)
    train_data, train_labels, val_data, val_labels, expected = random_split(ratio=0.1, study_name=study, rand=False)
    model = svm.SVC(C=1, kernel='rbf', class_weight={1: (1 - numpy.mean(train_labels))  / numpy.mean(train_labels)})
    # train the model
    clf = model.fit(numpy.nan_to_num(train_data), numpy.nan_to_num(train_labels))
    y_pred = clf.predict(numpy.nan_to_num(val_data))
    print(y_pred)
    x_pred = clf.predict(numpy.nan_to_num(train_data))
    compute_metrics(svm_total, val_labels.flatten(), y_pred, train_labels, x_pred)
for key in svm_total:
    ave = numpy.asarray(svm_total[key]).mean(axis=0)
    print('{0}: {1}'.format(key, ave))

In [ ]:
print(train_data.shape)
print(val_data.shape)

# nearest neigbour classifier

In [ ]:
train_data, train_labels, val_data, val_labels, expected = random_split()
print(train_data.shape)
print(val_data.shape)

In [ ]:
def predict(train_data, validation_data, train_labels):
    tmp = []
    for i in range(len(validation_data)):
        diff = train_data - validation_data[i]
        idx = numpy.argmin(numpy.sqrt(numpy.sum(diff ** 2, axis=1)))
        tmp.append(idx)
    return train_labels[tmp]

In [ ]:
y_pred = predict(numpy.nan_to_num(train_data), numpy.nan_to_num(val_data), train_labels)
x_pred = predict(numpy.nan_to_num(train_data), numpy.nan_to_num(train_data), train_labels)

In [ ]:
nearest_total = defaultdict(list)
compute_metrics(nearest_total, val_labels.flatten(), y_pred, train_labels, x_pred)
for key in nearest_total:
    print('{0}: {1}'.format(key, nearest_total[key][-1]))

# moses

In [ ]:
from opencog.atomspace import AtomSpace
from opencog.pymoses import moses
from opencog.scheme_wrapper import scheme_eval

In [ ]:
train_data, train_labels, val_data, val_labels = next(split('study_16446_GPL570_all-bmc15'))

In [ ]:
input_data = numpy.concatenate([train_labels, train_data], axis=1)

In [ ]:
input_data[:,[0, 2]] = input_data[:,[2,0]]

In [ ]:
min(0, 179) / max(107, 0)

In [ ]:
mos = moses()

In [ ]:
output = mos.run(input=input_data, python=True, args='--balance=1 -m 100000')

In [ ]:
output[0].program

In [ ]:
mos = moses()
input_data = [[0, 0, 0], [1, 1, 0], [1, 0, 1], [2, 1, 1]]
output = mos.run(input=input_data, python=True)
print (output[0].score) # Prints: 0
model = output[0].eval
print(model([0, 1]))  # Returns: True
print(model([1, 1]))  # Returns: False